**Bloco 1: Ingestão de Dados e Conexão Cloud**

Este bloco foca na sua habilidade de conectar ferramentas de nuvem de forma segura.

In [ ]:
"""
MÓDULO: Ingestão de Dados via GCP
AUTORA: Sueli da Hora (Analytics Engineer)
DESCRIÇÃO: Conexão segura com a Feature Store no BigQuery e carga de dataset refinado.
"""

from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Autenticação de escopo de usuário para recursos do Google Cloud SDK
auth.authenticate_user()

# Inicialização do cliente BigQuery para o projeto FlightOnTime
PROJECT_ID = 'flightontime-hackathon'
client = bigquery.Client(project=PROJECT_ID)

# Query otimizada: Consumindo apenas a tabela de treinamento refinada (5GB brutos já tratados)
QUERY_REFINADA = """
SELECT * FROM `flightontime-hackathon.dados_voos.tabela_treinamento_final`
"""
df = client.query(QUERY_REFINADA).to_dataframe()

# Validação de integridade do DataFrame carregado em memória
print(f"Ingestão concluída. Volume: {df.shape[0]} registros processados.")
df.head()

Ingestão concluída. Volume: 6982766 registros processados.


,companhia,origem,estado_origem,destino,estado_destino,distancia,atrasado,data_voo,hora_partida_prevista
0,OH,ABE,Pennsylvania,CLT,North Carolina,481.0,0,2024-04-23,1148
1,OH,ABE,Pennsylvania,CLT,North Carolina,481.0,0,2024-02-11,1202
2,OH,ABE,Pennsylvania,CLT,North Carolina,481.0,0,2024-12-01,601
3,OH,ABE,Pennsylvania,CLT,North Carolina,481.0,0,2024-10-17,815
4,G4,ABE,Pennsylvania,MYR,South Carolina,518.0,0,2024-06-30,1933


**Bloco 2: Pipeline de Machine Learning**

Este bloco mostra que você entende como transformar dados brutos em inteligência preditiva

In [ ]:
"""
MÓDULO: Engine de Predição (Machine Learning)
MODELO: Random Forest Classifier (Acurácia: 80.13%)
AUTORA: Sueli da Hora
"""

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib

# 1. Feature Engineering: Extração de sazonalidade temporal
df['data_voo'] = pd.to_datetime(df['data_voo'])
df['mes'] = df['data_voo'].dt.month
df['dia_semana'] = df['data_voo'].dt.dayofweek

# 2. Encoding: Normalização de variáveis categóricas para processamento matemático
le = LabelEncoder()
target_cols = ['companhia', 'origem', 'estado_origem', 'destino', 'estado_destino']
for col in target_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# 3. Data Splitting: Divisão estratégica entre treinamento e validação (80/20)
X = df.drop(columns=['atrasado', 'data_voo'])
y = df['atrasado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Fit do Modelo: Aplicação de Random Forest com controle de profundidade para evitar overfitting
modelo = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
modelo.fit(X_train, y_train)

# 5. Avaliação de Performance: Extração de métricas de acurácia e recall
y_pred = modelo.predict(X_test)
print(f"Acurácia Validada: {accuracy_score(y_test, y_pred):.2%}")

# Exportação do binário do modelo para deploy no Back-end
joblib.dump(modelo, 'modelo_atraso_voo.joblib')

Acurácia Validada: 80.03%


['modelo_atraso_voo.joblib']

**Bloco 3: Persistência e Governança**

Este bloco garante que o arquivo final esteja sob sua posse no Drive.

In [ ]:
from google.colab import drive
import shutil

"""
MÓDULO: Persistência de Ativos
OBJETIVO: Backup do modelo treinado no Google Drive pessoal.
"""

from google.colab import drive
import shutil

# Montagem do File System do Google Drive
drive.mount('/content/drive')

# Migração do artefato .joblib para armazenamento permanente
DESTINO_DRIVE = '/content/drive/MyDrive/modelo_atraso_voo.joblib'
shutil.copy('modelo_atraso_voo.joblib', DESTINO_DRIVE)

print(f"Modelo persistido com sucesso em: {DESTINO_DRIVE}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Modelo persistido com sucesso em: /content/drive/MyDrive/modelo_atraso_voo.joblib
